In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hashlib
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from sklearn.ensemble import RandomForestRegressor
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.optimize import minimize
#from plotnine import *
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from joblib import dump

In [2]:
train_data = pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/train.csv")
train_updates = pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")
test_data = pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/test.csv")
test_target = pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/test_labels.csv")

In [23]:
train_data.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [24]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


In [3]:
train_data["pH"] = train_data["pH"].fillna(train_data["pH"].mean())

In [4]:
train_data["data_source"] = train_data["data_source"].fillna(train_data["data_source"].mode()[0])

In [27]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31390 non-null  float64
 3   data_source       31390 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


In [5]:
train_data.to_csv("train_data.csv", index=False)

In [6]:
# Define the hashing function
def hash_func(text):
    return int(hashlib.sha256(text.encode('utf-8')).hexdigest(), 16) % 10**8

In [7]:
train_data["protein_sequence"] = train_data["protein_sequence"].apply(hash_func)

In [8]:
train_data["data_source"] = train_data["data_source"].apply(hash_func)

In [10]:
train_data.head(10)

,seq_id,protein_sequence,pH,data_source,tm
0,0,27542999,7.0,59999825,75.7
1,1,90790636,7.0,59999825,50.5
2,2,58836494,7.0,59999825,40.5
3,3,49513385,7.0,59999825,47.2
4,4,12660678,7.0,59999825,49.5
5,5,38645583,7.0,59999825,48.4
6,6,42461684,7.0,59999825,45.7
7,7,13370569,7.0,59999825,55.9
8,8,47311580,7.0,59999825,48.1
9,9,72191155,7.0,59999825,49.7


In [11]:
train_data["pH"].value_counts()

7.000000     26462
7.500000       576
8.000000       502
3.000000       288
6.892339       286
             ...  
3.140000         1
2.530000         1
2.810000         1
39.000000        1
10.300000        1
Name: pH, Length: 116, dtype: int64

In [9]:
features = train_data.drop("tm", axis = 1)
target = train_data["tm"]

In [25]:
#y_binary = (target > 0.5).astype(int)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

from tensorflow.keras.callbacks import EarlyStopping

# create an instance of EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [11]:
test_data["protein_sequence"] = test_data["protein_sequence"].apply(hash_func)
test_data["data_source"] = test_data["data_source"].apply(hash_func)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


num_samples = 1000
seq_len = 10
num_features = 5
# Define the RNN model architecture
model = keras.Sequential(
    [
        layers.LSTM(64, input_shape=(X_train.shape[1], 1)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1),
    ]
)

# Compile the model
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on the testing data
loss, mae = model.evaluate(X_test, y_test)

# Print the mean absolute error
print(f"Mean absolute error: {mae}")

In [103]:
# split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a Random Forest model and fit it to the training data
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# evaluate the performance of the Random Forest model using cross-validation
scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mean_mse = -np.mean(scores)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", mean_mse)

# create a Gradient Boosting model and fit it to the training data
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=42)
gb_model.fit(X_train, y_train)

# evaluate the performance of the Gradient Boosting model using cross-validation
scores = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mean_mse = -np.mean(scores)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", mean_mse)

Cross-validation scores: [-136.18017103 -135.27246659 -124.57937144 -125.76985538 -127.61174709]
Mean cross-validation score: 129.88272230354286
Cross-validation scores: [-133.10504774 -131.91555384 -122.09401286 -123.7763546  -126.70212862]
Mean cross-validation score: 127.51861953327132


In [114]:
# Save the trained model to a file
dump(gb_model, 'GradientBoostingRegressor.joblib')

['GradientBoostingRegressor.joblib']

In [104]:
y_pred = gb_model.predict(test_data)

In [105]:
y_pred

array([71.03467708, 67.76561582, 66.99060471, ..., 64.24265654,
       69.35423305, 68.23345368])

# Build the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Train a Random Forest classifier
clf = RandomForestRegressor(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = clf.score(X_test, y_test)
print("Test accuracy:", accuracy)


test_data.info()

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from math import sqrt
dtrain = xgb.DMatrix(X_train, label=y_train)

# Define the hyperparameters
params = {
    'max_depth': 5,
    'eta': 0.1,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'objective': 'reg:squarederror'
}

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=1000)

dtest = xgb.DMatrix(test_data, label=test_target["tm"])

y_pred = model.predict(dtest)

y_pred

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

regressor = XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.15, max_delta_step=None, max_depth=5,
             min_child_weight=3, monotone_constraints=None,
             n_estimators=500, n_jobs=16, num_parallel_tree=None,
             random_state=0, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
             subsample=None, tree_method=None, validate_parameters=1,eval_metric=smape,
             verbosity=None)

regressor.fit(X_train, y_train, 
             early_stopping_rounds=10, 
             eval_set=[(X_test, y_test)],
             verbose=False)

y_pred = regressor.predict(test_data)

y_pred

y_pred = model.predict(test_data)

y_pred

y_pred = clf.predict(test_data)

y_pred

In [15]:
test_path = "/kaggle/input/novozymes-enzyme-stability-prediction/test.csv"
test_df = pd.read_csv(test_path)
seq_id = []
for i in range(len(test_df)):
    line = test_df.iloc[i]
    seq_id.append(line["seq_id"])

In [107]:
tm = []
#start_time = time.time()
for i in range(len(test_df)):
    prob = y_pred[i]
    tm.append(prob)

In [108]:
submission = pd.DataFrame(data={"seq_id": seq_id, "tm": tm})
#submission = submission.sort_values("row_id", ascending=False)

In [109]:
submission.to_csv("submission2.csv", index=False)
submission

,seq_id,tm
0,31390,71.034677
1,31391,67.765616
2,31392,66.990605
3,31393,70.765644
4,31394,71.034677
...,...,...
2408,33798,69.714542
2409,33799,69.366411
2410,33800,64.242657
2411,33801,69.354233


In [110]:
submission["tm"].value_counts()

69.559437    114
70.213801     95
69.714542     60
71.034677     51
69.218771     46
            ... 
60.212668      1
62.567911      1
64.496794      1
76.754797      1
69.354233      1
Name: tm, Length: 424, dtype: int64

In [111]:
from sklearn.metrics import mean_squared_error

# Compute the mean squared error of the predictions
mse = mean_squared_error(test_target["tm"], submission["tm"])
print(f"Mean squared error: {mse}")

Mean squared error: 80.32017661493623


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor


# Define the individual models
#logreg_model = LogisticRegression()
xgb_model = XGBRegressor()
gb_model = GradientBoostingRegressor()

# Create the VotingRegressor pipeline
pipeline = Pipeline([
    # Add any necessary preprocessing steps here, such as scaling or encoding
    
    ('ensemble', VotingRegressor([
        #('logreg', logreg_model),
        ('xgb', xgb_model),
        ('gb', gb_model)
    ]))
])


In [26]:
params = {
    #'ensemble__logreg__C': [0.1, 1, 10],  # Regularization parameter for Logistic Regression
     # Solver algorithm options for Logistic Regression
    'ensemble__xgb__max_depth': [3, 6, 9],  # Maximum tree depth for XGBRegressor
    'ensemble__xgb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate for XGBRegressor
    'ensemble__xgb__n_estimators': [100, 200, 300],  # Number of trees for XGBRegressor
    'ensemble__gb__max_depth': [3, 6, 9],  # Maximum tree depth for GradientBoostingRegressor
    'ensemble__gb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate for GradientBoostingRegressor
    'ensemble__gb__n_estimators': [100, 200, 300]  # Number of trees for GradientBoostingRegressor
}


In [ ]:
grid = GridSearchCV(pipeline, params, cv=5)
grid.fit(X_train, y_train)

In [ ]:
best_model = grid.best_estimator_
best_params = grid.best_params_

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor


In [13]:
logreg_pipeline = Pipeline([
    # Add any necessary preprocessing steps here, such as scaling or encoding
    
    ('logreg', LogisticRegression())
])


In [14]:
xgb_pipeline = Pipeline([
    # Add any necessary preprocessing steps here, such as scaling or encoding
    
    ('xgb', XGBRegressor())
])


In [15]:
gb_pipeline = Pipeline([
    # Add any necessary preprocessing steps here, such as scaling or encoding
    
    ('gb', GradientBoostingRegressor())
])


In [40]:
logreg_params = {
    'logreg__C': [0.1, 1, 10],  # Regularization parameter
    'logreg__solver': ['liblinear', 'saga']  # Solver algorithm options
}

xgb_params = {
    'xgb__max_depth': [3, 6, 9],  # Maximum tree depth
    'xgb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate
    'xgb__n_estimators': [100, 200, 300]  # Number of trees
}

gb_params = {
    'gb__max_depth': [3, 6, 9],  # Maximum tree depth
    'gb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate
    'gb__n_estimators': [100, 200, 300]  # Number of trees
}


In [18]:
from sklearn.ensemble import VotingRegressor




In [36]:
ensemble = VotingRegressor([
    ('xgb', xgb_pipeline),
    ('gb', gb_pipeline)
], weights=None)


In [41]:
params = {
    'xgb__xgb__max_depth': [3, 6, 9],  # Maximum tree depth for XGBRegressor
    'xgb__xgb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate for XGBRegressor
    'xgb__xgb__n_estimators': [100, 200, 300],  # Number of trees for XGBRegressor
    'gb__gb__max_depth': [3, 6, 9],  # Maximum tree depth for GradientBoostingRegressor
    'gb__gb__learning_rate': [0.1, 0.01, 0.001],  # Learning rate for GradientBoostingRegressor
    'gb__gb__n_estimators': [100, 200, 300]  # Number of trees for GradientBoostingRegressor
}


In [ ]:
grid = GridSearchCV(ensemble, params, cv=5)
grid.fit(X_train, y_train)


In [ ]:
best_model = grid.best_estimator_
best_params = grid.best_params_


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.preprocessing import FunctionTransformer


In [24]:
# Thresholding function to convert regressor outputs into classes
def threshold_predictions(y_pred, threshold=0.5):
    return np.where(y_pred >= threshold, 1, 0)

# Instantiate the regressor models
xgb_model = XGBRegressor()
gb_model = GradientBoostingRegressor()

# Define the models with thresholding
xgb_classifier = Pipeline([
    ('xgb', xgb_model),
    ('threshold', FunctionTransformer(threshold_predictions))
])

gb_classifier = Pipeline([
    ('gb', gb_model),
    ('threshold', FunctionTransformer(threshold_predictions))
])


TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...)' (type <class 'xgboost.sklearn.XGBRegressor'>) doesn't

In [ ]:
# Instantiate the Logistic Regression model
logreg_model = LogisticRegression()

# Create the VotingClassifier pipeline
pipeline = Pipeline([
    # Add any necessary preprocessing steps here, such as scaling or encoding
    
    ('ensemble', VotingClassifier([
        ('logreg', logreg_model),
        ('xgb', xgb_classifier),
        ('gb', gb_classifier)
    ]))
])


In [ ]:
params = {
    'ensemble__logreg__C': [0.1, 1, 10],  # Regularization parameter for Logistic Regression
    'ensemble__xgb__xgb__max_depth': [3, 6, 9],  # Maximum tree depth for XGBRegressor
    'ensemble__xgb__threshold__kw_args': [{'threshold': t} for t in [0.3, 0.5, 0.7]],  # Threshold for XGBRegressor
    'ensemble__gb__gb__max_depth': [3, 6, 9],  # Maximum tree depth for GradientBoostingRegressor
    'ensemble__gb__threshold__kw_args': [{'threshold': t} for t in [0.3, 0.5, 0.7]]  # Threshold for GradientBoostingRegressor
}


In [ ]:
grid = GridSearchCV(pipeline, params, cv=5)
grid.fit(X_train, y_train)